In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import cv2

import os

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def read_img(img_scr, size=False):
    img = cv2.imread(img_scr)
    if size:
        img = cv2.resize(img, (size,size))
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [ ]:
PATH = '../input/shopee-product-matching'
TRAIN_PATH = 'train_images'
TEST_PATH = 'test_images'

N = 25

In [ ]:
train = pd.read_csv(PATH+'/train.csv')
imgs_id = train['image'][:N]

In [ ]:
plt.figure(figsize=(20,20))

for i,img_scr in enumerate(imgs_id):
    img = read_img(os.path.join(PATH, TRAIN_PATH, img_scr))
    plt.subplot(np.ceil(N**.5), np.ceil(N**.5), i+1)
    plt.imshow(img)
    plt.axis('off')

In [ ]:
image = read_img(os.path.join(PATH, TRAIN_PATH, img_scr))

In [ ]:
from albumentations import *

def augment_and_show(aug, image):
    image_aug = aug(image=image)['image']
    imgs = [image,image_aug]
    name = ['Real image', 'Image after aug']
    
    plt.figure(figsize=(20, 20))
    for i,img in enumerate(imgs):
        plt.subplot(1,2,i+1)
        plt.imshow(img)
        plt.title(name[i], size=20)
        plt.axis('off')

In [ ]:
aug = HorizontalFlip(p=1.0)
augment_and_show(aug, image)

In [ ]:
aug = IAAPerspective(scale=(0.05, 0.1), p=1)
augment_and_show(aug, image)

In [ ]:
aug = ShiftScaleRotate(shift_limit=.1,
    scale_limit=.2,
    rotate_limit=30,
    interpolation=0,
    border_mode=2,
    value=0,
    mask_value=None,
    always_apply=False,
    p=1)

augment_and_show(aug, image)

In [ ]:
aug = RandomRotate90(p=1)

augment_and_show(aug, image)

In [ ]:
aug = Transpose(p=1)

augment_and_show(aug, image)

In [ ]:
aug = Blur(blur_limit=(5,10),p=1)

augment_and_show(aug, image)

In [ ]:
aug = MedianBlur(blur_limit=(5,9),p=1)

augment_and_show(aug, image)

In [ ]:
aug = RandomBrightnessContrast(brightness_limit=(.8,.9),
    contrast_limit=0,
    brightness_by_max=False,
    p=1
)

augment_and_show(aug, image)

In [ ]:
aug = CLAHE(clip_limit=4.0, tile_grid_size=(8, 8),p=1)
augment_and_show(aug, image)

In [ ]:
aug = IAAEmboss(alpha=(0.2, 0.5), strength=(2, 7),p=1)

augment_and_show(aug, image)

In [ ]:
def strong_aug(p=.5):
    return Compose([
        RandomRotate90(),
        Flip(),
        Transpose(),
        OneOf([
            IAAAdditiveGaussianNoise(),
            GaussNoise(),
        ], p=0.2),
        OneOf([
            MotionBlur(p=.2),
            MedianBlur(blur_limit=3, p=0.1),
            Blur(blur_limit=3, p=0.1),
        ], p=0.2),
        ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.2),
        OneOf([
            OpticalDistortion(p=0.3),
            GridDistortion(p=.1),
            IAAPiecewiseAffine(p=0.3),
        ], p=0.2),
        OneOf([
            CLAHE(clip_limit=2),
            IAASharpen(),
            IAAEmboss(),
            RandomBrightnessContrast(),            
        ], p=0.3),
        HueSaturationValue(p=0.3),
    ], p=p)

In [ ]:
aug = strong_aug(p=1)
augment_and_show(aug, image)